In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.color import gray2rgb
import cv2
from pandas import read_csv
import matplotlib.pyplot as plt

%matplotlib inline

## Net construction

In [ ]:
from fd import get_file_list, nnet_three_conv_layer
from plotting import plot_face_bb

In [ ]:
nnet = nnet_three_conv_layer()

In [ ]:
train_folder = 'train/'

In [ ]:
fnames, bboxes = get_file_list(train_folder)

In [ ]:
fnames[0], bboxes[0]

In [ ]:
X = np.array(list(fnames))
y = np.array(list(bboxes))

In [ ]:
nnet.fit(X, y)

In [ ]:
plot_face_bb(fnames[1000], bboxes[1000])

In [ ]:
im = imread(fnames[0])

In [ ]:
X = []
X.append(np.transpose(im, [2,0,1]))
X.append(None)
X.append(np.transpose(im, [2,0,1]))
X.append(None)

In [ ]:
np.array(X).shape

In [ ]:
plt.imshow(imread('/home/arya_03/data/concat/data/0/bb008b9e367f001837b3adabb2b46323.jpg'))

## Visualization

In [ ]:
from evaluate import plot_learning_curves, load_network
import cPickle as pickle
from models import nnet_three_conv_layer
from utils import get_file_list

In [ ]:
model_name = 'nnet.pickle'

In [ ]:
train_folder = 'train/'

In [ ]:
nnet = load_network(model_name)

In [ ]:
fnames, bboxes = get_file_list(train_folder)

In [ ]:
nnet.predict